In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import torch
from sentence_transformers import SentenceTransformer, util

# os.environ["TRANSFORMERS_CACHE"] = "./transformers_models"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
PLM_name = "BAAI/bge-m3"
PLM_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(PLM_name, cache_folder='../../../huggingface_models/hub').to(device)

/home/rl/conda/envs/LLM_Muti_Label/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
import json

In [3]:
id2information = json.load(open(f'./label_description.json', 'r', encoding='utf-8'))
idx2id = {i:k for i, (k, v) in enumerate(id2information.items())}

In [4]:
import json
import sys
sys.path.append('../../')
from llm.utils.prompt_utils import Prompt_hepler

ph = Prompt_hepler()
prompts = {
    'desc_dict': '{"code": "{code}", "description": "{description}"}',
}

prompt_id = 'desc_dict'
prompt = prompts[prompt_id]

In [5]:
informations = []

for k, v in id2information.items():
    d = {'code': k, 'description': v}
    vv = msg = ph.replace_with_dict(prompt, d, '{', '}')
    informations.append(str(vv))

In [6]:
informations_embeddings = (model.encode(informations, show_progress_bar=True, convert_to_tensor=True))

Batches: 100%|██████████| 2980/2980 [00:37<00:00, 79.88it/s] 


In [7]:
import random
from tqdm import tqdm

In [8]:
torch.save({
    'idx2id': idx2id,
    'informations_embeddings': informations_embeddings.cpu()
}, f'informations_embeddings_{"bge" if PLM_name=="BAAI/bge-m3" else "MiniLM"}.pt')